# libs & configs

In [1]:
from rapidfuzz import process, fuzz
import unidecode
import re
import calendar
import pandas as pd
import xlsxwriter
from openpyxl import load_workbook
import os
import sys
from pandas import to_datetime as pdDatetime
from datetime import datetime
from dateutil.relativedelta import relativedelta
from pandas import to_numeric as pdNumeric
from collections import defaultdict


In [2]:
# pega o diretorio acima de noteboosks, ou seja o do projeto principal
# current_dir = os.path.dirname(os.path.abspath(__file__)) - # isso nao funciona no jupyter , pois o __file__ e exclusivo de scrpit py
current_dir = os.getcwd()

# muda o diretorio para o projeto --
# isso é necessario para que o python consiga importar os modulos do projeto
# caso contrario, ele nao consegue achar os modulos
project_root = os.path.join(current_dir, '..')
sys.path.append(project_root)

In [3]:
from Scripts.functions import GetAno, mapear_vendedores_por_similaridade


# 00 - montando base de informaçoes

## Regioes 

In [4]:
# Criar os dados com abreviações
estados_regiões = {
    "Estado": [
        "Acre", "Alagoas", "Amapá", "Amazonas", "Bahia", "Ceará", "Distrito Federal",
        "Espírito Santo", "Goiás", "Maranhão", "Mato Grosso", "Mato Grosso do Sul",
        "Minas Gerais", "Pará", "Paraíba", "Paraná", "Pernambuco", "Piauí",
        "Rio de Janeiro", "Rio Grande do Norte", "Rio Grande do Sul", "Rondônia",
        "Roraima", "Santa Catarina", "São Paulo", "Sergipe", "Tocantins"
    ],
    "Cd Uf": [
        "AC", "AL", "AP", "AM", "BA", "CE", "DF",
        "ES", "GO", "MA", "MT", "MS",
        "MG", "PA", "PB", "PR", "PE", "PI",
        "RJ", "RN", "RS", "RO", "RR",
        "SC", "SP", "SE", "TO"
    ],
    "Região": [
        "Norte", "Nordeste", "Norte", "Norte", "Nordeste", "Nordeste", "Centro-Oeste",
        "Sudeste", "Centro-Oeste", "Nordeste", "Centro-Oeste", "Centro-Oeste",
        "Sudeste", "Norte", "Nordeste", "Sul", "Nordeste", "Nordeste",
        "Sudeste", "Nordeste", "Sul", "Norte", "Norte",
        "Sul", "Sudeste", "Nordeste", "Norte"
    ]
}

## clientes °

In [5]:
# -- 1 lendo arquivos
df_clients_bling = pd.read_excel(r"..\Data\01-01-23_08-07-25\sh-clientes_bling.xlsx")
df_clients_infosoft = pd.read_excel(r"..\Data\01-01-23_08-07-25\sh-clientes_infosoft.xlsx")

# -- 2 - padroniza os nomes para maior facilidade
df_clients_infosoft.rename(columns={'Cgc Cpf':'Clie Cgc Cpf'}, 
                                inplace=True)
df_clients_bling.rename(columns={'CPF/CNPJ': 'Clie Cgc Cpf',
                                  'Nome':'Descrição', 
                                  'Endereço': 'Endereco',
                                  'UF':'Cd Uf', 
                                  'Cidade':'Nm Cidade', 
                                  'Bairro':'Bairro',
                                'Cliente desde':'Dt Cadastro', 
                                'Email':'E Mail'
                                }, inplace=True)

# -- 3 - faz o full join 
df_clientes_merge = pd.merge(df_clients_infosoft,df_clients_bling,
                                  on='Clie Cgc Cpf',   # chave primaria
                                  how='outer',    # full outer join no pandas       
                                  suffixes=('_infosoft','_bling')) # renomeias o sufixo das colunas (col1_df1,col2_df2...)

# -- 4 - filtrando para todos os clients ou seja '''full join com priorização do df1'''
cols = ['Descrição', 'Endereco', 'Cd Uf','Nm Cidade', 'Bairro', 'Dt Cadastro','E Mail'] # colunas que quero manter (na esquerda)
for col in cols: 
    df_clientes_merge[col] = df_clientes_merge[f'{col}_infosoft'].combine_first(df_clientes_merge[f'{col}_bling']) # where quando a esquerda for vazia colca conteudo da direita, colca tudo na nova coluna 
    
df_clientes_masquerade = df_clientes_merge[['Clie Cgc Cpf'] + cols] # atribui a df final o conteudo completo nas colunas novas sem esquecer da chave primeira



## nfs ~


In [6]:
'''
corrigir cod vendedores bling
nr de pedido

'''


# -- 1 - le os arquivos
df_nfs_bling = pd.read_excel(r"..\Data\01-01-23_08-07-25\sh-nfs_bling.xlsx")
df_nfs_infosoft = pd.read_excel(r"..\Data\01-01-23_08-07-25\sh-nfs_infosoft.xlsx")

# -- 2 - padroniza as colunas para evitar complicações
df_nfs_bling.rename(columns={'Data de emissão':'Dt. emissão', 
                             'Data de Saída/Entrada':'Dt Sai Ent', 
                             'Número':'Nr. nota', 
                             'CNPJ/CPF':'Clie Cgc Cpf',
                             'Código do vendedor':'Cd Vendedor', 
                             'Desconto':'Vl Total Desc', 
                             'Valor total líquido':'Vl Total Nota'
                             },inplace=True)

# -- 3 -  junta as duas df 
df_nfs_masquerade = pd.concat([df_nfs_bling,df_nfs_infosoft], axis=0)

# # -- 4 - arrumando vendedores !!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!
# df_nfs_masquerade['Cd Vendedor'] = df_nfs_masquerade['Cd Vendedor'].fillna('').astype(str).str.strip()

# df_nfs_masquerade, logs = mapear_vendedores_por_similaridade(df_nfs_masquerade, df_vendedores_masquerade)


# -- 5 - padroniza os tipos e filtr
df_nfs_masquerade['Clie Cgc Cpf'] = (
    df_nfs_masquerade['Clie Cgc Cpf']
    .astype(str)
    .str.strip()                  # remove espaços antes/depois
    .str.replace(r'\s+', '', regex=True)  # remove espaços internos
)
ice = ['27.191.195/0001-50',
'18.848.204/0001-42',
'08.913.661/0001-10',
 '11.990.266/0001-45',
 '05.012.654/0001-59',
 '04.466.924/0001-39',
 '44.667.686/0001-44',
 '45.385.860/0001-29',
 '41.400.310/0001-80' ]
df_nfs_masquerade = df_nfs_masquerade[~df_nfs_masquerade['Clie Cgc Cpf'].isin(ice)]
df_nfs_masquerade['Dt Sai Ent'] = pdDatetime(df_nfs_masquerade['Dt Sai Ent'] , dayfirst=True)
df_nfs_masquerade['Dt. emissão'] = pdDatetime(df_nfs_masquerade['Dt. emissão'], dayfirst=True)
df_nfs_masquerade['Vl Total Desc'] = df_nfs_masquerade['Vl Total Desc'].astype(str).str.replace(',','')
df_nfs_masquerade['Vl Total Nota'] = df_nfs_masquerade['Vl Total Nota'].astype(str).str.replace(',','')
df_nfs_masquerade['Vl Total Desc'] = pdNumeric(df_nfs_masquerade['Vl Total Desc'],errors='coerce')
df_nfs_masquerade['Vl Total Nota'] =  pdNumeric(df_nfs_masquerade['Vl Total Nota'],errors='coerce')
df_nfs_masquerade['Nr Pedido'] =  pdNumeric(df_nfs_masquerade['Nr Pedido'],errors='coerce')



## pedidos

In [7]:
df_pedidos_infosoft = pd.read_excel(r"../Data/01-01-23_08-07-25/sh-pedidos_infosoft.xlsx")
df_pedidos_bling = pd.read_excel(r'..\Data\sh-pedidos-bling.xlsx')

df_pedidos_bling.rename(columns={'CPF/CNPJ':'CNPJ - CPF', 
                                      'Valor total': 'Vl. total',
                                      'Data': 'Dt. pedido',
                                      'N煤mero do pedido': 'Nr. pedido'
                                      },inplace=True)

df_pedidos_masquerade = pd.concat([df_pedidos_infosoft,df_pedidos_bling], axis=0)

df_pedidos_masquerade.rename(columns={'CNPJ - CPF':'Clie Cgc Cpf', 
                                      'Vl. total': 'Vl venda',
                                      'Nr. pedido': 'Nr Pedido',
                                      'Dt. pedido': 'Dt. emissão'
                                      },inplace=True)

# -- 5 - padroniza os tipos e filtr
df_pedidos_masquerade['Clie Cgc Cpf'] = (
    df_pedidos_masquerade['Clie Cgc Cpf']
    .astype(str)
    .str.strip()                  # remove espaços antes/depois
    .str.replace(r'\s+', '', regex=True)  # remove espaços internos
)
ice = ['27.191.195/0001-50',
'18.848.204/0001-42',
'08.913.661/0001-10',
 '11.990.266/0001-45',
 '05.012.654/0001-59',
 '04.466.924/0001-39',
 '44.667.686/0001-44',
 '45.385.860/0001-29',
 '41.400.310/0001-80' ]
df_pedidos_masquerade = df_pedidos_masquerade[~df_pedidos_masquerade['Clie Cgc Cpf'].isin(ice)]

df_pedidos_masquerade['Nr Pedido'] = pdNumeric(df_pedidos_masquerade['Nr Pedido'])
df_pedidos_masquerade['Vl venda'] = df_pedidos_masquerade['Vl venda'].astype(str).str.replace(',','')
df_pedidos_masquerade['Vl venda'] = pdNumeric(df_pedidos_masquerade['Vl venda'])
df_pedidos_masquerade['Dt. emissão'] =  pd.to_datetime(df_pedidos_masquerade['Dt. emissão'], dayfirst=True, )

print(df_pedidos_masquerade['Dt. emissão'].sort_values())


26     2023-07-03
20     2023-07-03
5      2023-07-04
6      2023-07-04
23     2023-07-05
          ...    
1469   2025-07-08
1468   2025-07-08
1466   2025-07-08
1462   2025-07-08
1467   2025-07-08
Name: Dt. emissão, Length: 1524, dtype: datetime64[ns]


## produtos

## vendedores °

In [8]:
vendedores_pix = pd.read_excel('..\Data\Posição representantes 2023 - 2024.xlsx', sheet_name='PIX REPRESENTANTES', skiprows=1)
vendedores_pix.rename(columns={'NOME':'Nm Vendedor'}, inplace=True)
vendedores_pix = vendedores_pix[vendedores_pix['Nm Vendedor'].notna()]

df_vendedores_masquerade = pd.read_excel(r'..\Data\01-01-23_08-07-25\sh-vendedores_infosoft.xlsx')
df_vendedores_masquerade = df_vendedores_masquerade[~pd.isna(df_vendedores_masquerade['Nm Vendedor'])]

df_vendedores_masquerade['Cd Vendedor'] = df_vendedores_masquerade['Cd Vendedor'].astype(float) 




# 01 - clientes novos por regiao por periodo ou vendedor



In [ ]:
''' mes / centro oeste / ...... / sul / total
total / ..... /  '''

In [68]:


# -- 1 - definindo dfs e colunas
df_clientes = df_clientes_masquerade[['Cd Uf','Nm Cidade','Dt Cadastro']]
df_estados_regiões = pd.DataFrame(estados_regiões)
df_novos = ['regiao', 'Nm cidade', 'Quant']

# -- 2 - definindo a região 
df_clientes = df_clientes.merge(df_estados_regiões[['Cd Uf', 'Região']],
                                on='Cd Uf', how='left' )

# -- 3 definindo periodo 
anos_meses = {2024: [1,2,3,4,5,6,8,9,10,11,12],
        2025: [1,2,3,4,5,6]}

# -- 4 - filtrando pelo peridodo
dados = [] # paera pord4er atribuir tudo depois 

for ano, meses in anos_meses.items():
    for mes in meses:
        primeiro_dia = datetime(ano, mes, 1)
        ultimo_dia = (primeiro_dia + relativedelta(months=1)) - relativedelta(days=1)
        
        df_novos_raw = df_clientes[
            (df_clientes['Dt Cadastro'] >= primeiro_dia) & 
            (df_clientes['Dt Cadastro'] <= ultimo_dia)
        ]

        for index, row in df_novos_raw.iterrows():
            dados.append({
                "Ano": ano,
                "Mês": mes,
                "Nm Cidade" : row['Nm Cidade'],
                "Região": row["Região"]
                 })

df_novos_raw = pd.DataFrame(dados)
df_novos_raw['Periodo'] = df_novos_raw['Ano'].astype(str) + '-' + df_novos_raw['Mês'].astype(str).str.zfill(2)

df_novos_2024 = df_novos_raw[df_novos_raw['Ano'] == 2024]
df_novos_2024.drop(columns=['Ano','Mês'], inplace=True)
df_novos_2024.groupby('Periodo')['Região'].count() .sum()

C:\Users\ADMIN\AppData\Local\Temp\ipykernel_4828\604163167.py:39: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_novos_2024.drop(columns=['Ano','Mês'], inplace=True)


np.int64(121)

In [20]:
# === 1. Leitura dos dados ===
df_sonia_emanu = pd.read_excel('emauel_sonia.xlsx')
# === 2. Garantir que a coluna de data está no formato datetime ===
df_nfs_masquerade['Dt. emissão'] = pd.to_datetime(df_nfs_masquerade['Dt. emissão'], errors='coerce')

# === 3. Definir a data limite para os "últimos 3 meses" (ex: a partir de fevereiro/2025) ===
data_limite = pd.Timestamp('2025-02-01')

# === 4. Identificar CPFs com nota fiscal emitida após a data limite ===
clientes_com_nf_recent = df_nfs_masquerade[
    df_nfs_masquerade['Dt. emissão'] >= data_limite
]['Clie Cgc Cpf'].unique()

# === 5. Filtrar df_sonia_emanu: manter apenas quem NÃO comprou após a data limite ===
df_sonia_emanu_sem_nf_recent = df_sonia_emanu[
    ~df_sonia_emanu['Cgc Cpf'].isin(clientes_com_nf_recent)
]

# === 6. Obter a data da última compra de cada cliente (mesmo que antiga) ===
ultima_compra = df_nfs_masquerade.groupby('Clie Cgc Cpf')['Dt. emissão'].max().reset_index()
ultima_compra.rename(columns={'Dt. emissão': 'Dt. última compra'}, inplace=True)

# === 7. Juntar a última compra no DataFrame filtrado ===
df_sonia_emanu_sem_nf_recent = df_sonia_emanu_sem_nf_recent.merge(
    ultima_compra,
    how='left',
    left_on='Cgc Cpf',
    right_on='Clie Cgc Cpf'
)

# === 8. Limpar colunas duplicadas do merge (opcional) ===
df_sonia_emanu_sem_nf_recent.drop(columns=['Clie Cgc Cpf'], inplace=True)

# === 9. Exportar para Excel ===
df_sonia_emanu_sem_nf_recent.to_excel('emauel_sonia2.xlsx', index=False)

In [92]:
# Caminho do relatório
path_relatorio = r"G:\Meu Drive\MASQUERADE\FINANCEIRO\RELATORIOS\CLIENTES-NOVOS\clientes_novos_região_08-2024_06-2025.xlsx"
workbook = xlsxwriter.Workbook(path_relatorio)
worksheet = workbook.add_worksheet('Relatório')

# Linha inicial
linha = 0

# Estilos
titulo = workbook.add_format({
    'bold': True, 
    'bg_color': "#A7A7A7", 
    'border': 6,
    'align': 'center', 'valign': 'vcenter'
})
cols = workbook.add_format({
    'border': 6, 
    'bold': True,
    'bg_color': "#72AC76", 
    'align': 'center', 'valign': 'vcenter'
})
infos = workbook.add_format({
    'border': 2, 
    'align': 'center', 'valign': 'vcenter'
})

# Cabeçalho principal
worksheet.merge_range("A1:H1", "RELATÓRIO DE CLIENTES NOVOS POR REGIÃO - 2024 - 2025", titulo)
linha += 2

# ---- Preparação dos dados

# Pivot
df_pivot = df_novos_raw.pivot_table(index='Periodo', columns='Região', aggfunc='size', fill_value=0)

# Garante todas as regiões
regioes = ['Centro-Oeste', 'Nordeste', 'Norte', 'Sudeste', 'Sul']
for reg in regioes:
    if reg not in df_pivot.columns:
        df_pivot[reg] = 0

df_pivot = df_pivot[regioes]
df_pivot['Total'] = df_pivot.sum(axis=1)
df_pivot.reset_index(inplace=True)

# Filtra por ano
df_2024 = df_pivot[df_pivot['Periodo'].str.startswith('2024')]
df_2025 = df_pivot[df_pivot['Periodo'].str.startswith('2025')]

headers = ['Período'] + regioes + ['Total']

# Função para escrever DataFrame
def escrever_df(worksheet, df, linha, format_info):
    for _, row in df.iterrows():
        for col, colname in enumerate(df.columns):
            worksheet.write(linha, col, row[colname], format_info)
        linha += 1
    return linha

# ---------- BLOCO 2024 ----------
worksheet.write(linha, 0, "2024", titulo)
linha += 1

for col, header in enumerate(headers):
    worksheet.write(linha, col, header, cols)
linha += 1

linha = escrever_df(worksheet, df_2024, linha, infos)

totais_2024 = df_2024[regioes + ['Total']].sum()
worksheet.write(linha, 0, 'Total 2024', cols)
for col, val in enumerate(totais_2024, start=1):
    worksheet.write(linha, col, val, infos)
linha += 2

# ---------- BLOCO 2025 ----------
worksheet.write(linha, 0, "2025", titulo)
linha += 1

for col, header in enumerate(headers):
    worksheet.write(linha, col, header, cols)
linha += 1

linha = escrever_df(worksheet, df_2025, linha, infos)

totais_2025 = df_2025[regioes + ['Total']].sum()
worksheet.write(linha, 0, 'Total 2025', cols)
for col, val in enumerate(totais_2025, start=1):
    worksheet.write(linha, col, val, infos)

# Finaliza o arquivo
workbook.close()
